In [1]:
import tensorflow as tf
import numpy as np
import math
from scipy.special import binom


def call(r, m, l):
    # r: (None, 3)
    r = tf.cast(r, dtype=tf.float32)
    r_norm = tf.norm(r, axis=1)
    l = tf.reshape(tf.cast(l, dtype=tf.float32), (1,))
    m = tf.reshape(tf.cast(m, dtype=tf.float32), (1,))
    
    prefactor = tf.reshape(tf.sqrt((l + 0.5) / math.pi), (1,))

    def factorial(x):
        x = tf.cast(x, dtype=tf.float32)
        return tf.exp(tf.math.lgamma(x + 1))
    
    def log_factorial(x):
        x = tf.cast(x, dtype=tf.float32)
        return tf.math.lgamma(x + 1)

    PI_ml = prefactor
    PI_ml *= tf.sqrt(factorial(l - m) / factorial(l + m))
    k = tf.range(int(tf.math.floor((l - m) / 2)) + 1)[None, :]
    k = tf.cast(k, dtype=tf.float32)
    b = k ** r
    # work in logarithmic scale
    fac = tf.cast(tf.math.floormod(k, 2), dtype=tf.float32)
    tf.debugging.check_numerics(fac, "nan at 1")
    fac -= l * tf.math.log(2.)
    fac += log_factorial(l) - log_factorial(l - k) - log_factorial(k)
    tf.debugging.check_numerics(fac, "nan at 2")
    fac += log_factorial(2 * l - 2 * k) - log_factorial(l - 2 * k) - log_factorial(l)
    tf.debugging.check_numerics(fac, "nan at 3")
    fac += log_factorial(l - 2 * k) - (l - 2 * k - m)
    tf.debugging.check_numerics(fac, "nan at 4")
    fac += (2 * k - 1) * (tf.math.log(r[:, 2]) - tf.math.log(r_norm))[:, None]
    tf.debugging.check_numerics(fac, "nan at 5")
    fac -= m * tf.math.log(r[:, 2, None])
    tf.debugging.check_numerics(fac, "nan at 6")
    fac = tf.math.exp(fac)
    PI_ml *= fac
    tf.debugging.check_numerics(fac, "nan in PI_ml")
    PI_ml = tf.reduce_sum(PI_ml, axis=1)

    if m == 0:
        res = PI_ml / tf.sqrt(2.)
    else:
        trigon = tf.sin if m < 0 else tf.cos
        m = tf.math.abs(m)
        m = tf.cast(m, dtype=tf.float32)[None, :]
        p = tf.range(tf.math.abs(m) + 1, dtype=tf.float32)[None, :]

        # work in logarithmic scale
        fac = log_factorial(m) - log_factorial(p) - log_factorial(m - p)
        fac += p * tf.math.log(r[:, 0, None]) 
        fac += (m - p) * tf.math.log(r[:, 1, None])
        fac += tf.math.log(trigon(m - p) * math.pi / 2)
        fac = tf.math.exp(fac)
        fac = tf.reduce_sum(fac, axis=1)
        res = PI_ml * fac
    return res

In [2]:
random_vector = np.random.normal(0, 1, (10, 3))
r = tf.constant(random_vector, dtype=tf.float32)

In [5]:
l = 0
l = tf.constant(l, dtype=tf.float32)
m = 0
m = tf.constant(m, dtype=tf.float32)
call(random_vector, m, l)

InvalidArgumentError: {{function_node __wrapped__CheckNumerics_device_/job:localhost/replica:0/task:0/device:CPU:0}} nan at 5 : Tensor had NaN values [Op:CheckNumerics]

In [6]:
float(1 < 0)

0.0